In [1]:
from osgeo import gdal
import numpy as np
import numexpr as ne
import os
import glob
import rasterio

In [2]:
DayDir = r'F:\MOD11A2_Day_Output\Output_Final_30k_2030pc'
NightDir = r'F:\MOD11A2_Night_Output\Output_Final_30k_2030pc'
DiffDir = r'G:\DiurnalDiffs'

In [3]:
DayFiles = glob.glob(os.path.join(DayDir,'*Data.tif'))
NightFiles = glob.glob(os.path.join(NightDir,'*Data.tif'))

In [4]:
len(DayFiles)

681

In [5]:
len(NightFiles)

681

In [6]:
for i in range (len(DayFiles)):
    dayFile = DayFiles[i]
    nightFile = NightFiles[i]
    assert dayFile.replace('Day','Night') == nightFile
    outBaseName = os.path.basename(dayFile).replace('Day', 'DiurnalDifference').replace('_Filled_Data','')
    outFile = os.path.join(DiffDir, outBaseName)
    #print outFile
    inFile = gdal.Open(dayFile)
    inBand = inFile.GetRasterBand(1)
    dayData = inBand.ReadAsArray()
    NDV = inBand.GetNoDataValue()
    inGT = inFile.GetGeoTransform()
    inProj = inFile.GetProjection()
    
    inFile = gdal.Open(nightFile)
    inBand = inFile.GetRasterBand(1)
    nightData = inBand.ReadAsArray()
   # where ((dayData == NDV) or (nightData == NDV), NDV)
    diffData = ne.evaluate("where (((dayData == NDV) | (nightData == NDV)), NDV, dayData - nightData)")
    
    outDrv = gdal.GetDriverByName("GTiff")
    outDS = outDrv.Create(outFile, diffData.shape[1], diffData.shape[0], 1, gdal.GDT_Float32,
                              ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
    outDS.SetGeoTransform(inGT)
    outDS.SetProjection(inProj)
    outBand = outDS.GetRasterBand(1)
    outBand.WriteArray(diffData)
    outBand.SetNoDataValue(NDV)
    outDS = None
    inFile = None
    #templateDS = None
    del diffData
    del dayData
    del nightData
